In [ ]:
!rm -rf cs484 && git clone https://github.com/berkan-sahin/cs484.git

Cloning into 'cs484'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 298 (delta 7), reused 14 (delta 5), pack-reused 282
Receiving objects: 100% (298/298), 83.49 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
!pip3 install -U torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import Tuple
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torchvision
from torchvision import models, transforms
from torch.autograd import Variable
from torchvision.io import read_image, ImageReadMode
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
import time
import copy





In [ ]:
class HockeyDataset(Dataset):

    def __init__(self, data_dir, transformer) -> None:
        self.data_dir = data_dir
        self.transformer = transformer

    def __len__(self):
        return 192

    def __getitem__(self, index) -> Tuple[any, any]:
        class_idx = index / 64
        if (class_idx < 1):
            class_name = 'freehit'
        elif (class_idx < 2):
            class_name = 'goal'
        elif (class_idx < 3):
            class_name = 'penaltycorner'
        else:
            class_name = 'penaltyshot'  # should never happen

        img_name = os.path.join(self.data_dir, class_name,
                                f'{(index % 64) + 1}.jpg')
        image = read_image(img_name).to(torch.float32)
        image = self.transformer(image)
        return image, class_idx


preprocess = transforms.Compose([
    # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.Resize(256, antialias=True),
    transforms.CenterCrop(224),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    # transforms.ToTensor(),
])

- modified the learning rate (from 0.0001 to 0.001)
- using pytorch's pretrained weights
- freezing the feature detection layers
- bugfixes
- batch size incremented to 32 from 4

In [ ]:
if True:    
    fold = 6
    epochs = 100
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device)
    #vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    vgg16 = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)
    # vgg16.load_state_dict(torch.load('vgg16.pth'))
    # reset the last layer
    # Freeze training for all layers
    for param in vgg16.features.parameters():
      param.require_grad = False

    in_features = vgg16.classifier[-1].in_features
    classifier = list(vgg16.classifier.children())[:-1]
    classifier.extend([nn.Linear(in_features, 3)])
    vgg16.classifier = nn.Sequential(*classifier)
    print(vgg16.classifier)
    vgg16 = vgg16.to(device)
    if torch.cuda.is_available():
        vgg16.cuda()
    dataset = HockeyDataset(
        'cs484/dataset', models.VGG16_Weights.IMAGENET1K_V1.transforms(antialias=True))
    # kfold = KFold(n_splits=fold, shuffle=True)
    kfold = StratifiedKFold(n_splits=fold, shuffle=True, random_state=3)

 #   train_size = int(0.8 * len(dataset))
 #   test_size = len(dataset) - train_size
 #   train, test = random_split(dataset, [train_size, test_size])

    begin = time.time()
    initial_weight = copy.deepcopy(vgg16.state_dict())
    accuracy = []
    best_epochs = []
    for (fold, (train, test)) in enumerate(kfold.split(dataset, np.full(64, 0).tolist() + np.full(64, 1).tolist() + np.full(64, 2).tolist())):
        print("Train: ", train, "Validation: ", test)
        trainloader = DataLoader(dataset, batch_size=40, sampler=train, num_workers=2)
        valloader = DataLoader(dataset, batch_size=40, sampler=test, num_workers=2)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(vgg16.parameters(), lr=0.001)
        #optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
        scheduler = optim.lr_scheduler.StepLR(
               optimizer, step_size=25, gamma=0.85)

        vgg16.load_state_dict(initial_weight)
        best_weight = copy.deepcopy(initial_weight)
        best_acc = 0.0
        best_epoch = 0
        for epoch in range(epochs):
            # Training phase
            vgg16.train()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in trainloader:
                labels = labels.type(torch.LongTensor)
                #inputs = inputs.to(device)
                #labels = labels.to(device)

                if torch.cuda.is_available():
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                optimizer.zero_grad()
                outputs = vgg16(inputs)
                _, preds = torch.max(outputs, 1)
                #print(type(labels))
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                scheduler.step()

                del inputs, outputs, labels, preds

            epoch_loss = running_loss / len(train)
            epoch_acc = running_corrects.double() / len(train)
            print('Epoch: {} train Loss: {:.4f} Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))
            running_loss = 0.0
            running_corrects = 0
            vgg16.eval()
            with torch.no_grad():
                for images, labels in valloader:
                    labels = labels.type(torch.LongTensor)
                    optimizer.zero_grad()
                    #images = images.to(device)
                    #labels = labels.to(device)

                    if torch.cuda.is_available():
                      images, labels = Variable(images.cuda()), Variable(labels.cuda())
                    else:
                      images, labels = Variable(images), Variable(labels)
                    outputs = vgg16(images)
                    _, predicted = torch.max(outputs.data, 1)
                    loss = criterion(outputs, labels)
                    running_loss += loss.item() * images.size(0)
                    running_corrects += torch.sum(predicted == labels.data)
            
            epoch_loss = running_loss / len(test)
            epoch_acc = running_corrects.double() / len(test)
            print('Epoch: {} eval Loss: {:.4f} Acc: {:.4f}'.format(epoch, epoch_loss, epoch_acc))

            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_weight = copy.deepcopy(vgg16.state_dict())
                best_epoch = epoch
        
        accuracy.append(best_acc)
        best_epochs.append(best_epoch)
        print(f"Fold {fold} accuracy: {best_acc} achieved in epoch {best_epoch}")
        #torch.save(best_weight, f"vgg16_fold{fold}.pth")


    time_elapsed = time.time() - begin
    print(f"Finished Training, took {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
    print(f"Average accuracy: {sum(accuracy) / len(accuracy)}")
    print(f"Average epoch: {sum(best_epochs) / len(best_epochs)}")

Using device:  cuda:0
Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)
Train:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  15  17  18  19  20
  21  22  23  24  25  26  27  29  30  31  32  33  35  37  39  40  41  42
  43  44  45  46  47  48  49  50  51  52  53  55  56  59  61  62  63  64
  65  67  69  71  72  74  75  76  77  78  79  80  81  84  86  87  88  89
  90  92  93  94  95  96  97  98  99 101 102 103 104 105 107 108 109 110
 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 131
 132 133 135 136 137 138 139 140 141 143 144 145 146 147 148 149 150 152
 153 154 155 156 157 159 160 161 162 163 164 165 166 167 169 170 171 172
 173 174 175 176 178 180 181 182 183 184 185 186 188 189 190 1

avg. accuracies (step size 7, gamma 0.1)
- 50 epochs, batch size 32, lr 0.001 : 0.4
- 50 epochs, batch size 32, lr 0.0005: 0.4
- 50 epochs, batch size 16, lr 0.001: 0.38
- VGG16_bn, 50 epochs, batch size 32, lr: 0.001: 0.43
- VGG16_bn, 50 epochs, batch size 32, lr: 0.004: 0.39

avg. accuracies (step size 10, gamma 0.05)
- VGG16_bn, 50 epochs, batch size 32, lr: 0.001: 0.56
- VGG16_bn, 100 epochs, batch size 32, lr: 0.001, step size: 15, gamma: 0.05: 0.49
- VGG16_bn, 50 epochs, batch size 32, lr: 0.001, no LR scheduler: 0.78
- VGG16_bn, 50 epochs, batch size 40, lr: 0.001, no LR scheduler [note: overfitting after epoch 26]: 0.807
- VGG16_bn, 50 epochs, batch size 40, lr: 0.001, step size: 25, gamma: 0.05: 0.67
- VGG16_bn, 50 epochs, batch size 40, lr: 0.001, step size: 25, gamma: 0.90: 0.807
- VGG16_bn, 50 epochs, batch size 40, lr: 0.001, step size: 25, gamma: 0.95: 0.807
- **VGG16_bn, 100 epochs, batch size 40, lr: 0.001, step size: 25, gamma: 0.95: 0.848**
- VGG16_bn, 50 epochs, batch size 40, lr: 0.001, step size: 25, gamma: 0.85: 0.76
